In [2]:
import pandas as pd
import scipy as sp
import numpy as np
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation
import xgboost as xgb
import pprint as pp
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
import bnp_helper
import common_helper
import random
import os
import datetime
from sklearn.externals import joblib
from matplotlib.pylab import rcParams
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier



In [3]:
################### Combine the training set and test set to fix the unmatched categories ############
train_df_raw = pd.read_csv("../data/train.csv")
test_df_raw = pd.read_csv("../data/test.csv")

df_raw_combined = pd.concat([train_df_raw, test_df_raw], axis = 0)

########### Clean and Impute (Combined) #############
df_combinded = bnp_helper.clean(df_raw_combined, drop_collinearity = True, inplace = True)
df_combinded = bnp_helper.impute_cate_with_na_numeric_with_outlier(df_combinded)

################ Convert text to number (Combined) ################
df_combinded = common_helper.dummify(df_combinded, bnp_helper.get_categorical_variables(df_combinded))
print df_combinded.shape
df_combinded.head()

(228714, 449)


ID  target          v1       v10        v101        v102        v103  \
0   3       1    1.335739  0.503281    8.389237    2.757375    4.374296   
1   4       1 -999.000000  1.312910 -999.000000 -999.000000 -999.000000   
2   5       1    0.943877  0.765864    5.879353    3.292788    5.924457   
3   6       1    0.797415  6.542669    8.507281    2.503055    4.872157   
4   8       1 -999.000000  1.050328 -999.000000 -999.000000 -999.000000   

         v106        v108        v109       v114        v117        v118  \
0   12.579184    2.382692    3.930922  15.634907    6.592012    5.909091   
1 -999.000000    1.825361    4.247858  10.308044   10.595357 -999.000000   
2   11.670572    1.375753    1.184211  11.205561    3.478911    6.233767   
3   12.554274    2.230754    1.990131  13.777666    4.949609    7.180722   
4 -999.000000 -999.000000 -999.000000  14.097099 -999.000000 -999.000000   

           v119        v120        v122        v123        v124        v126  \
0 -6.297423e-07    1.059603    8.000000    1.989780    0.035754    1.804126   
1 -9.990000e+02 -999.000000 -999.000000 -999.000000    0.598896 -999.000000   
2 -2.792745e-07    2.138728    9.333333    2.477596    0.013452    1.773709   
3  5.655086e-01    1.166281    7.018256    1.812795    0.002267    1.415230   
4 -9.990000e+02 -999.000000 -999.000000 -999.000000 -999.000000 -999.000000   

         v127  v129         v13        v130        v131        v14  \
0    3.113719     0    2.866830    0.636365    2.857144  11.636387   
1 -999.000000     0 -999.000000 -999.000000 -999.000000  11.636386   
2    3.922193     2    2.505589    0.883118    1.176472   9.603542   
3    2.954381     1    3.903302    1.677108    1.034483  14.094723   
4 -999.000000     0 -999.000000 -999.000000 -999.000000  10.991098   

          v16         v18         v19          v2       v21           v23  \
0    8.571429    0.106720    0.148883    8.727474  7.730923 -1.716131e-08   
1 -999.000000 -999.000000 -999.000000 -999.000000  6.763110 -9.990000e+02   
2    5.882353    0.244541    0.144258    5.310079  5.245035 -2.785053e-07   
3    5.517242    1.224114    0.231630    8.304757  7.517125 -4.805344e-07   
4 -999.000000 -999.000000 -999.000000 -999.000000  6.414567 -9.990000e+02   

          v27         v28         v35         v36         v37  v38  \
0    3.393503    0.590122    8.375452   11.326592    0.454546    0   
1 -999.000000 -999.000000 -999.000000   14.579479 -999.000000    0   
2    5.306122    0.836005    7.959184   12.730517    0.259740    0   
3    2.303640    8.926662    8.898468   11.302795    0.433735    0   
4 -999.000000 -999.000000 -999.000000 -999.000000 -999.000000    0   

           v4         v42         v43         v44         v45         v49  \
0    3.921026   12.707581    2.015505   10.498338    9.848672    8.086643   
1 -999.000000 -999.000000 -999.000000 -999.000000 -999.000000 -999.000000   
2    4.410969   12.346939    2.926830    8.897561    5.343819    6.836734   
3    4.225930   12.935823    1.470878   12.708574    9.670823    8.591954   
4 -999.000000 -999.000000 -999.000000 -999.000000 -999.000000 -999.000000   

        v50         v51         v53         v55         v57         v58  \
0  0.899420    7.277792   16.747968    1.299638    3.971118    0.529802   
1  1.379210 -999.000000 -999.000000 -999.000000 -999.000000 -999.000000   
2  0.604504    9.637627   15.102041    0.765305    4.030613    4.277456   
3  3.329176    4.780357   16.621695    1.178161    3.965517    1.732102   
4  1.364536 -999.000000 -999.000000 -999.000000 -999.000000 -999.000000   

          v59          v6         v60         v61  v62         v63  \
0   10.890984    2.599278    1.588448   15.858152    1    0.153461   
1 -999.000000 -999.000000 -999.000000 -999.000000    2    2.544736   
2    9.105481    3.979592    2.151361   16.075602    1    0.123643   
3   11.777912    2.097700    1.229246   15.927390    1    0.140260   
4 -999.000000 -999.000000 -999.000000 -999.000000  

In [5]:
train_df = df_combinded[-df_raw_combined['target'].isnull()]
test_df = df_combinded[df_raw_combined['target'].isnull()]

#train_df_sample = train_df.sample(50000, random_state = 0)
train_df_sample = train_df
target_train = train_df_sample['target']
X_train = train_df_sample.drop(['ID', 'target'], axis = 1)

In [6]:
############# load the saved models ###########
model_logit = joblib.load('save/logit_model.pkl')
model_xgb = joblib.load('save/xgb_model_2.pkl')
model_extratree = joblib.load('save/extratree/extratree_model.pkl')

In [9]:
############ Try ensemble voting with average ##########
ensemble_avg = VotingClassifier(estimators=[('lr', model_logit), ('xgb', model_xgb), ('extratree', model_extratree)], 
                                  voting='soft', weights=[1,1,1])

start = datetime.datetime.now()
ensemble_avg.fit(X_train, target_train)
end = datetime.datetime.now()
print "model training time: {}".format(end - start)

model training time: 0:48:42.957426


In [11]:
predprob_train = ensemble_avg.predict_proba(X_train)
metrics.log_loss(target_train, predprob_train)

0.43721866001258147

In [ ]:
joblib.dump(ensemble_avg, 'save/ensemble/ensemble_avg_model.pkl')

In [15]:
################# Predict the test set and save result to file ############
#test_df_sample = test_df.sample(1000, random_state = 0)
test_df_sample = test_df

X_test = test_df_sample.drop(['ID', 'target'], axis = 1)

#Predict test set:
predprob_test = ensemble_avg.predict_proba(X_test)
        
# Save results
ids = test_df_sample['ID']
predprob_one_test = [p[1] for p in predprob_test]
bnp_helper.save_result(ids, predprob_one_test, "results/ensemble_avg_result.csv")

print 'Done at: {}'.format(datetime.datetime.now())

Done at: 2016-04-15 14:56:12.388888


In [16]:
############ ensemble voting with weighted ##########
ensemble_weighted = VotingClassifier(estimators=[('lr', model_logit), ('xgb', model_xgb), ('extratree', model_extratree)], 
                                  voting='soft', weights=[1,3,2])

start = datetime.datetime.now()
ensemble_weighted.fit(X_train, target_train)
end = datetime.datetime.now()
print "model training time: {}".format(end - start)

model training time: 0:34:28.962945


In [29]:
predprob_train = ensemble_weighted.predict_proba(X_train)
print metrics.log_loss(target_train, predprob_train)

joblib.dump(ensemble_weighted, 'save/ensemble/ensemble_weighted_model.pkl')

0.424929884275


In [19]:
################# Predict the test set and save result to file ############
test_df_sample = test_df

X_test = test_df_sample.drop(['ID', 'target'], axis = 1)

#Predict test set:
predprob_test = ensemble_weighted.predict_proba(X_test)
        
# Save results
ids = test_df_sample['ID']
predprob_one_test = [p[1] for p in predprob_test]
bnp_helper.save_result(ids, predprob_one_test, "results/ensemble_weighted_result.csv")

print 'Done at: {}'.format(datetime.datetime.now())

Done at: 2016-04-15 16:06:23.664569


In [32]:
############ ensemble voting with weighted ##########
ensemble_weighted_2 = VotingClassifier(estimators=[('xgb', model_xgb), ('extratree', model_extratree)], 
                                  voting='soft', weights=[7,3])

start = datetime.datetime.now()
ensemble_weighted_2.fit(X_train, target_train)
end = datetime.datetime.now()
print "model training time: {}".format(end - start)

predprob_train = ensemble_weighted_2.predict_proba(X_train)
log_loss = metrics.log_loss(target_train, predprob_train)
print "log loss: {}".format(log_loss)

model training time: 0:33:19.581539
log loss: 0.413489377317


In [28]:
################# Predict the test set and save result to file ############
test_df_sample = test_df

X_test = test_df_sample.drop(['ID', 'target'], axis = 1)

#Predict test set:
predprob_test = ensemble_weighted_2.predict_proba(X_test)
        
# Save results
ids = test_df_sample['ID']
predprob_one_test = [p[1] for p in predprob_test]
bnp_helper.save_result(ids, predprob_one_test, "results/ensemble_weighted_2_result.csv")

print 'Done at: {}'.format(datetime.datetime.now())

Done at: 2016-04-15 19:16:17.931736
